In [ ]:
#part 1 in Decision Tree 

In [16]:
header = ["color", "diameter", "label"]
def unique_vals(rows, col):
    #Find the unique values for a column in a dataset#
    return set([row[col] for row in rows])

def class_counts(rows):
    """Counts the number of each type of example in a dataset."""
    counts = {}  # a dictionary of label -> count.
    for row in rows:
        # in our dataset format, the label is always the last column
        label = row[-1]
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts
def is_numeric(value):
    """Test if a value is numeric."""
    return isinstance(value, int) or isinstance(value, float)

In [68]:
training_data = [
    ['Orange', 'Large', 'Basketball'],
    ['White', 'Small', 'Golfball'],
    ['Green', 'Medium', 'Tennis'],
    ['White', 'Small', 'Golfball'],
    ['Orange', 'Large', 'Lemon'],
]

In [69]:
class Question:
    def __init__(self, column, value):
        self.column = column
        self.value = value
    
    def match(self, example):
        val = example[self.column]
        if is_numeric(val):
            return val >= self.value
        else:
            return val == self.value
    
    def __repr__(self):
        # This is just a helper method to print
        # the question in a readable format.
        condition = "=="
        if is_numeric(self.value):
            condition = ">="
        return "Is %s %s %s?" % (
            header[self.column], condition, str(self.value))

In [70]:
#a numeric question ---> is our value greater than or equal to 3
Question(0, 'orange')
#categorical question ---> is our value green 
q = Question(0, 'Medium')

In [71]:
def partition(rows, question):
    true_rows, false_rows = [], []
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows

In [72]:
#splits the data at the node
true_rows, false_rows = partition(training_data, Question(0, 'Red'))
print(true_rows)
print(false_rows)

[]
[['Orange', 'Large', 'Basketball'], ['White', 'Small', 'Golfball'], ['Green', 'Medium', 'Tennis'], ['White', 'Small', 'Golfball'], ['Orange', 'Large', 'Lemon']]


In [73]:
#Quantifying uncertainty.... lets us know what question will reduce uncertainty the most
#creates the function for this
def gini(rows):
    counts = class_counts(rows)
    impurity = 1
    for lbl in counts:
        prob_of_lbl = counts[lbl] / float(len(rows))
        impurity -= prob_of_lbl**2
    return impurity

def info_gain(left, right, current_uncertainty):
    p = float(len(left)) / (len(left) + len(right))
    return current_uncertainty - p * gini(left) - (1 - p) * gini(right)

In [74]:
def find_best_split(rows):
    best_gain = 0  
    best_question = None 
    current_uncertainty = gini(rows)
    n_features = len(rows[0]) - 1 
    
    for col in range(n_features): 
        values = set([row[col] for row in rows])
        for val in values:  
            question = Question(col, val)
            true_rows, false_rows = partition(rows, question)
            if len(true_rows) == 0 or len(false_rows) == 0:
                continue
            gain = info_gain(true_rows, false_rows, current_uncertainty)

            if gain > best_gain:
                best_gain, best_question = gain, question

    return best_gain, best_question


In [75]:
best_gain, best_question = find_best_split(training_data)
print(best_question)

Is color == White?


In [76]:
class Leaf:
    def __init__(self, rows):
        self.predictions = class_counts(rows)

In [77]:
class Decision_Node: 
    def __init__(self,
                 question,
                 true_branch,
                 false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch

In [78]:
def build_tree(rows):
    gain, question = find_best_split(rows)
    if gain == 0:
        return Leaf(rows)
    true_rows, false_rows = partition(rows, question)

    true_branch = build_tree(true_rows)

    false_branch = build_tree(false_rows)

    return Decision_Node(question, true_branch, false_branch)

In [79]:
def print_tree(node, spacing=""):
    """World's most elegant tree printing function."""
# Base case: we've reached a leaf
    if isinstance(node, Leaf):
        print (spacing + "Predict", node.predictions)
        return
    
    print (spacing + str(node.question))
    
    print (spacing + '--> True:')
    print_tree(node.true_branch, spacing + "  ")
    
    print (spacing + '--> False:')
    print_tree(node.false_branch, spacing + "  ") 
    
my_tree = build_tree(training_data)
    
print_tree(my_tree)
    

Is color == White?
--> True:
  Predict {'Golfball': 2}
--> False:
  Is color == Orange?
  --> True:
    Predict {'Basketball': 1, 'Lemon': 1}
  --> False:
    Predict {'Tennis': 1}


In [80]:
def classify(row, node):
    """See the 'rules of recursion' above."""

    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        return node.predictions

    # Decide whether to follow the true-branch or the false-branch.
    # Compare the feature / value stored in the node,
    # to the example we're considering.
    if node.question.match(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)

In [81]:
def print_leaf(counts):
    """A nicer way to print the predictions at a leaf."""
    total = sum(counts.values()) * 1.0
    probs = {}
    for lbl in counts.keys():
        probs[lbl] = str(int(counts[lbl] / total * 100)) + "%"
    return probs

In [94]:
testing_data = [
    ['White', "Small", 'Golf'],
    ['White', "Small", 'Golf'],
    ['Green', "Medium", 'Tennis'],
    ['Green', "Medium", 'Tennis'],]

In [95]:
for row in testing_data:
    print ("Actual: %s. Predicted: %s" %
           (row[-1], print_leaf(classify(row, my_tree))))

Actual: Golf. Predicted: {'Golfball': '100%'}
Actual: Golf. Predicted: {'Golfball': '100%'}
Actual: Tennis. Predicted: {'Tennis': '100%'}
Actual: Tennis. Predicted: {'Tennis': '100%'}


In [96]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt # data visualization
import seaborn as sns # statistical data visualization
%matplotlib inline

In [97]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [98]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [100]:
data = '/kaggle/input/car-evaluation-data-set/car_evaluation.csv'

In [102]:
df= pd.read_csv(data)

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/car-evaluation-data-set/car_evaluation.csv'